In [1]:
from os import system
system('pip install pycryptodome')

0

In [2]:
from ipywidgets import interact, interact_manual, widgets,fixed
import wave
from IPython.display import display, Audio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import h5py
from copy import copy
from os import system,path,remove
from Crypto.Cipher import AES

In [3]:
system('wget patrickhaddad.xyz/data/traces/ex01.h5')

0

In [4]:
hdf5_file="ex01.h5"
root_name='enonce_0'
data_file = h5py.File(hdf5_file,'r')
m_Y=data_file[root_name+ "/traces"][:,150:]
m_cipher=data_file[root_name+ "/ciphertexts"][:,:]
v_iv=data_file[root_name+ "/iv"][:]
data_file.close()

In [5]:
v_Y_std=m_Y.std(0)
v_index_nz=np.where(v_Y_std!=0)[0]
m_Y=m_Y[:,v_index_nz]
v_Y_mean=m_Y.mean(0)
v_Y_std=m_Y.std(0)
nb_pts=m_Y.shape[1]
nb_traces=m_Y.shape[0]

In [6]:
v_HW=np.zeros((256,),dtype=np.float64)
for i in range(256):
    v_HW[i]=bin(i).count('1')
v_rcon=np.array([0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a,0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39,0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a,0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8,0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef,0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc,0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b,0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3,0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94,0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20,0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35,0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f,0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04,0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63,0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd,0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb],dtype=np.uint8)
v_SBox=np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16 ],dtype=np.uint8)
v_SBoxInv=np.array([0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d],dtype=np.uint8)

In [7]:
log_screen=widgets.Text(value='',description='', disabled=True,layout=widgets.Layout(width='256px'),font_size=12)

In [20]:
def write_python_script(v_key_in):
    fid=open('python_file_to_run.py','w')
    s_line="import numpy as np\nfrom Crypto.Cipher import AES\nimport h5py\n"
    fid.write(s_line)

    s_line='data_file = h5py.File("ex01.h5","r")\nm_cipher=data_file["enonce_0/ciphertexts"][:,:]\nv_iv=data_file["enonce_0/iv"][:]\ndata_file.close()\n'
    fid.write(s_line)

    s_line='v_rcon=np.array([0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a,0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39,0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a,0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8,0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef,0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc,0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b,0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3,0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94,0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20,0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35,0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd, 0x61, 0xc2, 0x9f,0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb, 0x8d, 0x01, 0x02, 0x04,0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d, 0x9a, 0x2f, 0x5e, 0xbc, 0x63,0xc6, 0x97, 0x35, 0x6a, 0xd4, 0xb3, 0x7d, 0xfa, 0xef, 0xc5, 0x91, 0x39, 0x72, 0xe4, 0xd3, 0xbd,0x61, 0xc2, 0x9f, 0x25, 0x4a, 0x94, 0x33, 0x66, 0xcc, 0x83, 0x1d, 0x3a, 0x74, 0xe8, 0xcb],dtype=np.uint8)\n'
    fid.write(s_line)

    s_line='v_SBox=np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16 ],dtype=np.uint8)\n'
    fid.write(s_line)

    s_line='v_best_key_in=['
    for i in range(16):
      s_line="%s0x%.2x,"%(s_line,v_key_in[i])
    s_line="%s]\n"%(s_line[:-1])
    fid.write(s_line)

    s_line="reversed_expandedKey = np.zeros(176,dtype=np.uint8)\nreversed_expandedKey[160:]=np.array(v_best_key_in,dtype=np.uint8)\nN=4\nR=11\n"
    fid.write(s_line)

    s_line="for i in range(N*R-1,3,-1):\n"
    fid.write(s_line)

    s_line="\tv_j=4*i+np.arange(4,dtype=np.uint8)\n\tif i%N==0:\n"
    fid.write(s_line)

    s_line="\t\trotw=reversed_expandedKey[4*(i-1)+(np.arange(1,5,dtype=np.uint8)%4)]\n\t\tsbw=v_SBox[rotw]\n\t\tsbw[0]^=v_rcon[int(i/N)]\n\t\treversed_expandedKey[v_j-16]=reversed_expandedKey[v_j]^sbw\n"
    fid.write(s_line)  

    s_line="\telse:\n"
    fid.write(s_line)  
        
    s_line="\t\treversed_expandedKey[v_j-16]=reversed_expandedKey[v_j]^reversed_expandedKey[v_j-4]\n"
    fid.write(s_line)          

    s_line="ret_key=reversed_expandedKey[:16]\nv_cipher=m_cipher.reshape(m_cipher.shape[0]*m_cipher.shape[1])\no_decrypt = AES.new(ret_key.tobytes(), AES.MODE_CBC,v_iv.tobytes())\nv_plain = np.frombuffer(o_decrypt.decrypt(v_cipher.tobytes()),dtype=np.uint8)\nnp.save('v_plain.npy',v_plain)\n"
    fid.write(s_line)              

    fid.close()

def pearson_attack_last_round(byte_to_attack=range(16),key_0=range(256),key_1=range(256),key_2=range(256),key_3=range(256),key_4=range(256),key_5=range(256),key_6=range(256),key_7=range(256),key_8=range(256),key_9=range(256),key_10=range(256),key_11=range(256),key_12=range(256),key_13=range(256),key_14=range(256),key_15=range(256),nb_tracesto_use=range(16,512)):
    log_screen.value='AUCUN FICHIER A LIRE'
    v_key_hyp=np.array([key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,key_12,key_13,key_14,key_15],dtype=np.uint8)
    write_python_script(v_key_hyp)
    m_x=np.zeros((nb_tracesto_use,16),dtype=np.uint8)
    for i in range(16):
        m_x[:,i]=v_SBoxInv[m_cipher[:nb_tracesto_use,i]^v_key_hyp[i]]
    v_x=v_HW[m_x[:,byte_to_attack]]
    m_Y_tmp=m_Y[:nb_tracesto_use,:]
    m_Y_tmp_mean=m_Y_tmp.mean(0)
    m_Y_tmp_std=m_Y_tmp.std(0)
    v_r2=np.zeros((nb_pts,),dtype=np.float64)
    v_denum=((m_Y[:nb_tracesto_use,:].std(0))*(v_x.std()))
    v_num=((np.transpose(np.broadcast_to(v_x,(nb_pts,nb_tracesto_use)))*m_Y_tmp).mean(0)-(m_Y_tmp_mean)*(v_x.mean()))
    v_nz=np.where(v_denum!=0)[0]
    v_r2[v_nz]=abs(v_num[v_nz]/v_denum[v_nz])
    
    plt.figure(figsize=[15,7])
    plt.plot(v_r2)
    plt.show()

def Run_Dec_read():
    if (path.exists('v_plain.npy')==False):
      log_screen.value='AUCUN FICHIER A LIRE'
    else:      
      remove('v_plain.npy')
    log_screen.value='CHARGEMENT EN COURS'
    system('python python_file_to_run.py')
    v_audio_frames_in=np.load('v_plain.npy')
    sounddata=np.frombuffer(v_audio_frames_in.tobytes(),dtype='int64')
    ret_audio=Audio(sounddata,rate=8000)
    ret_audio.reload()
    ret_audio.autoplay=True
    display(ret_audio)

    log_screen.value='FICHIER PRET'

In [18]:
print('********************* MESSAGE DE LA PLUS HAUTE IMPORTANCE *********************\n')
print('La variable m_cipher est un message qui est protegee par un AES 128\n')
print('Pour casser cette protection il faut tester les\n 340282366920938463463374607431768211456 possibilitÃ©es\n')
print('Je sais seulement qu il s agit encore d un fichier .wav \n')
print('J ai recupere la conso du circuit pendant le chiffrement \n')
print('Tu peux tester la valeur de la cle et tracer le coef de pearson de la derniere Sbox\n')
print('Une fois la protection cassÃ©e, tu pourras Ã©couter la musique en utilisant la fonction try_key_and_play([K0,K1,K2,..,K15])\n')
print('Elle a un seul parametre qui est la cle secrete\n')

print('Allez je compte sur toi. On m a dit que t es le meilleur pour faire ce job\n')

********************* MESSAGE DE LA PLUS HAUTE IMPORTANCE *********************

La variable m_cipher est un message qui est protegee par un AES 128

Pour casser cette protection il faut tester les
 340282366920938463463374607431768211456 possibilitÃ©es

Je sais seulement qu il s agit encore d un fichier .wav 

J ai recupere la conso du circuit pendant le chiffrement 

Tu peux tester la valeur de la cle et tracer le coef de pearson de la derniere Sbox

Une fois la protection cassÃ©e, tu pourras Ã©couter la musique en utilisant la fonction try_key_and_play([K0,K1,K2,..,K15])

Elle a un seul parametre qui est la cle secrete

Allez je compte sur toi. On m a dit que t es le meilleur pour faire ce job



In [38]:
im=interact(pearson_attack_last_round)
im.widget.close()
im.widget.children[0].max=15
im.widget.children[0].min=0
im.widget.children[17].max=512
im.widget.children[17].min=4
im.widget.children[17].value=128
im.widget.children[17].layout=widgets.Layout(width='150px')
im.widget.children[0].layout=widgets.Layout(width='150px')
for i in range(16):
    im.widget.children[i+1].max=15
    im.widget.children[i+1].min=0
    im.widget.children[i+1].layout=widgets.Layout(width='150px')

v_vb1=[]
out = widgets.Output()
with out:
  im2=interact_manual(Run_Dec_read)
  im2.widget.children[0].description='Dechiffrer et Ecouter'
v_vb1.append(out)

up_box=widgets.HBox((im.widget.children[0],im.widget.children[17],log_screen,v_vb1[0]))
keyl_box=widgets.HBox((im.widget.children[1],im.widget.children[2],im.widget.children[3],im.widget.children[4],im.widget.children[5],im.widget.children[6],im.widget.children[7],im.widget.children[8]))
keyh_box=widgets.HBox((im.widget.children[9],im.widget.children[10],im.widget.children[11],im.widget.children[12],im.widget.children[13],im.widget.children[14],im.widget.children[15],im.widget.children[16]))
all_box=widgets.VBox([up_box,keyl_box,keyh_box,im.widget.children[18]])

display(all_box)

interactive(children=(Dropdown(description='byte_to_attack', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…